# Spelling Correction

<div class="alert alert-info">

This tutorial is available as an IPython notebook at [Malaya/example/spell-correction](https://github.com/huseinzol05/Malaya/tree/master/example/spell-correction).
    
</div>

In [1]:
%%time
import malaya

CPU times: user 5.37 s, sys: 1.03 s, total: 6.4 s
Wall time: 7.18 s


In [2]:
# some text examples copied from Twitter

string1 = 'krajaan patut bagi pencen awal skt kpd warga emas supaya emosi'
string2 = 'Husein ska mkn aym dkat kampng Jawa'
string3 = 'Melayu malas ni narration dia sama je macam men are trash. True to some, false to some.'
string4 = 'Tapi tak pikir ke bahaya perpetuate myths camtu. Nanti kalau ada hiring discrimination despite your good qualifications because of your race tau pulak marah. Your kids will be victims of that too.'
string5 = 'DrM cerita Melayu malas semenjak saya kat University (early 1980s) and now as i am edging towards retirement in 4-5 years time after a career of being an Engineer, Project Manager, General Manager'
string6 = 'blh bntg dlm kls nlp sy, nnti intch'

### Load probability speller

The probability speller extends the functionality of the Peter Norvig's, http://norvig.com/spell-correct.html.

And improve it using some algorithms from Normalization of noisy texts in Malaysian online reviews, https://www.researchgate.net/publication/287050449_Normalization_of_noisy_texts_in_Malaysian_online_reviews.

Also added custom vowels and consonant augmentation to adapt with our local shortform / typos.

```python
def probability(sentence_piece: bool = False, **kwargs):
    """
    Train a Probability Spell Corrector.

    Parameters
    ----------
    sentence_piece: bool, optional (default=False)
        if True, reduce possible augmentation states using sentence piece.

    Returns
    -------
    result: malaya.spell.Probability class
    """
```

In [3]:
prob_corrector = malaya.spell.probability()

#### To correct a word

```python
def correct(self, word: str, **kwargs):
    """
    Most probable spelling correction for word.

    Parameters
    ----------
    word: str

    Returns
    -------
    result: str
    """
```

In [4]:
prob_corrector.correct('sy')

'saya'

In [5]:
prob_corrector.correct('mhthir')

'mahathir'

In [6]:
prob_corrector.correct('mknn')

'makanan'

#### List possible generated pool of words

```python
def edit_candidates(self, word):
    """
    Generate candidates given a word.

    Parameters
    ----------
    word: str

    Returns
    -------
    result: List[str]
    """
```

In [7]:
prob_corrector.edit_candidates('mhthir')

['mahathir']

In [8]:
prob_corrector.edit_candidates('smbng')

['sembang',
 'smbg',
 'sambung',
 'simbang',
 'sembung',
 'sumbang',
 'sambong',
 'sambang',
 'sumbing',
 'sombong',
 'sembong']

Now you can see, `edit_candidates` suggested quite a lot candidates and some of candidates not an actual word like `sambang`, to reduce that, we can use [sentencepiece](https://github.com/google/sentencepiece) to check a candidate a legit word for malaysia context or not.

In [10]:
prob_corrector_sp = malaya.spell.probability(sentence_piece = True)
prob_corrector_sp.edit_candidates('smbng')

['sumbing',
 'sambung',
 'smbg',
 'sembung',
 'sombong',
 'sembong',
 'sembang',
 'sumbang',
 'sambong']

**So how does the model knows which words need to pick? highest counts from the corpus!**

#### To correct a sentence

```python
def correct_text(self, text: str):
    """
    Correct all the words within a text, returning the corrected text.

    Parameters
    ----------
    text: str

    Returns
    -------
    result: str
    """
```

In [9]:
prob_corrector.correct_text(string1)

'kerajaan patut bagi pencen awal sakit kepada warga emas supaya emosi'

In [10]:
prob_corrector.correct_text(string2)

'Husein suka makan ayam dekat kampung Jawa'

In [11]:
prob_corrector.correct_text(string3)

'Melayu malas ini narration dia sama sahaja macam men are trash. True to some, false to some.'

In [12]:
prob_corrector.correct_text(string4)

'Tapi tak fikir ke bahaya perpetuate myths macam itu. Nanti kalau ada hiring discrimination despite your good qualifications because of your race tahu pula marah. Your kids will be victims of that too.'

In [13]:
prob_corrector.correct_text(string5)

'DrM cerita Melayu malas semenjak saya kat University (early 1980s) and now as saya am edging towards retirement in 4-5 years time after a career of being an Engineer, Project Manager, General Manager'

In [14]:
prob_corrector.correct_text(string6)

'boleh bintang dalam kelas nlp saya, nanti intch'

### Load JamSpell speller

JamSpell use Norvig + Ngram words.

Before you able to use this spelling correction, you need to install [jamspell](https://github.com/bakwc/JamSpell),

For mac,

```bash
wget http://prdownloads.sourceforge.net/swig/swig-3.0.12.tar.gz
tar -zxf swig-3.0.12.tar.gz
./swig-3.0.12/configure && make && make install
pip3 install jamspell
```

For debian / ubuntu,

```bash
apt install swig3
pip3 install jamspell
```

```python
def jamspell(model: str = 'wiki+news', **kwargs):
    """
    Load a jamspell Spell Corrector for Malay.

    Parameters
    ----------
    model: str, optional (default='wiki+news')
        Supported models. Allowed values:

        * ``'wiki+news'`` - Wikipedia + News, 337MB.
        * ``'wiki'`` - Wikipedia, 148MB.
        * ``'news'`` - Wikipedia, 215MB.

    Returns
    -------
    result: malaya.spell.JamSpell class
    """
```

In [4]:
model = malaya.spell.jamspell(model = 'wiki')

#### To correct a word

```python
def correct(self, word: str, string: str, index: int = -1):
    """
    Correct a word within a text, returning the corrected word.

    Parameters
    ----------
    word: str
    string: str
        Entire string, `word` must a word inside `string`.
    index: int, optional(default=-1)
        index of word in the string, if -1, will try to use `string.index(word)`.

    Returns
    -------
    result: str
    """
```

In [5]:
model.correct('suke', 'saya suke makan iyom')

'suka'

#### List possible generated pool of words

```python
def edit_candidates(self, word: str, string: str, index: int = -1):
    """
    Generate candidates given a word.

    Parameters
    ----------
    word: str
    string: str
        Entire string, `word` must a word inside `string`.
    index: int, optional(default=-1)
        index of word in the string, if -1, will try to use `string.index(word)`.

    Returns
    -------
    result: List[str]
    """
```

In [15]:
model.edit_candidates('ayem', 'saya suke makan ayem')

('ayem',
 'ayam',
 'ayer',
 'aye',
 'asem',
 'yem',
 'adem',
 'alem',
 'aem',
 'ayim',
 'oyem',
 'ayew',
 'azem',
 'ajem',
 'ayiem')

#### To correct a sentence

```python
def correct_text(self, text: str):
    """
    Correct all the words within a text, returning the corrected text.

    Parameters
    ----------
    text: str

    Returns
    -------
    result: str
    """
```

In [17]:
model.correct_text('saya suke makan ayom')

'saya suka makan ayam'

### Load Spylls speller

Spylls is Hunspell ported to Python.

Before you able to use this spelling correction, you need to install [spylls](https://github.com/zverok/spylls),

```bash
pip3 install Spylls
```

```python
def spylls(model: str = 'libreoffice-pejam', **kwargs):
    """
    Load a spylls Spell Corrector for Malay.

    Parameters
    ----------
    model : str, optional (default='libreoffice-pejam')
        Model spelling correction supported. Allowed values:

        * ``'libreoffice-pejam'`` - from LibreOffice pEJAm, https://extensions.libreoffice.org/en/extensions/show/3868

    Returns
    -------
    result: malaya.spell.Spylls class
    """
```

In [17]:
model = malaya.spell.spylls()

#### To correct a word

```python
def correct(self, word: str):
    """
    Correct a word within a text, returning the corrected word.

    Parameters
    ----------
    word: str

    Returns
    -------
    result: str
    """
```

In [18]:
model.correct('sy')

'st'

In [19]:
model.correct('mhthir')

'Mahathir'

In [20]:
model.correct('mknn')

'knn'

#### List possible generated pool of words

```python
def edit_candidates(self, word: str):
    """
    Generate candidates given a word.

    Parameters
    ----------
    word: str

    Returns
    -------
    result: List[str]
    """
    return list(self._dictionary.suggest(word))
```

In [21]:
model.edit_candidates('mhthir')

['Mahathir']

In [22]:
model.edit_candidates('smbng')

['sbng', 'smbang', 'jmbng', 'cmbng']

#### To correct a sentence

```python
def correct_text(self, text: str):
    """
    Correct all the words within a text, returning the corrected text.

    Parameters
    ----------
    text: str

    Returns
    -------
    result: str
    """
```

In [23]:
model.correct_text(string1)

'kerajaan putat baji pencen awal tks dpk warga enas supaya emisi'

In [24]:
model.correct_text(string2)

'Husein sak mkn aum tkad kampang Jawa'

In [25]:
model.correct_text(string3)

'Melayu malas in paration ida asma ja macam man ara tras. True tu som, falsafah tu som.'

In [26]:
model.correct_text(string4)

'Tapi kat fikir ka bahaya terperbuat smythea catu. Nanti kalau ada giring diskriminatif desiliter our food identification becus of our reca tua pukal ramah. Your kias wila ba victoria of rhat oto.'

### Load Encoder transformer speller

This spelling correction is a transformer based, improvement version of `malaya.spell.probability`. Problem with `malaya.spell.probability`, it naively picked highest probability of word based on public sentences (wiki, news and social media) without understand actual context, example,

```python
string = 'krajaan patut bagi pencen awal skt kpd warga emas supaya emosi'
prob_corrector = malaya.spell.probability()
prob_corrector.correct_text(string)
-> 'kerajaan patut bagi pencen awal sakit kepada warga emas supaya emosi'
```

It supposely replaced `skt` with `sikit`, a common word people use in social media to give a little bit of attention to `pencen`. So, to fix that, we can use Transformer model! **Right now transformer speller supported `BERT`, `ALBERT` and `ELECTRA` only**.

```python
def transformer_encoder(model, sentence_piece: bool = False, **kwargs):
    """
    Load a Transformer Encoder Spell Corrector. Right now only supported BERT, ALBERT and ELECTRA.

    Parameters
    ----------
    sentence_piece: bool, optional (default=False)
        if True, reduce possible augmentation states using sentence piece.

    Returns
    -------
    result: malaya.spell.Transformer class
    """
```

In [3]:
model = malaya.transformer.load(model = 'electra')
transformer_corrector = malaya.spell.transformer_encoder(model, sentence_piece = True)


Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use `tf.random.categorical` instead.






INFO:tensorflow:Restoring parameters from /Users/huseinzolkepli/Malaya/electra-model/base/electra-base/model.ckpt


#### To correct a sentence

```python
def correct_text(self, text: str, batch_size: int = 20):
    """
    Correct all the words within a text, returning the corrected text.

    Parameters
    ----------
    text: str
    batch_size: int, optional(default=20)
        batch size to insert into model.

    Returns
    -------
    result: str
    """
```

In [4]:
transformer_corrector.correct_text(string1)

'kerajaan patut bagi pencen awal sikit kepada warga emas supaya emosi'

perfect! But again, transformer model is very expensive! You can compare the time wall with probability based.

In [6]:
%%time
transformer_corrector.correct_text(string1)

CPU times: user 21.8 s, sys: 1.19 s, total: 23 s
Wall time: 5.15 s


'kerajaan patut bagi pencen awal sikit kepada warga emas supaya emosi'

In [7]:
%%time
prob_corrector.correct_text(string1)

CPU times: user 108 ms, sys: 3.34 ms, total: 112 ms
Wall time: 112 ms


'kerajaan patut bagi pencen awal sakit kepada warga emas supaya emosi'

### Load symspeller speller

This spelling correction is an improvement version for [symspeller](https://github.com/mammothb/symspellpy) to adapt with our local shortform / typos. Before you able to use this spelling correction, you need to install [symspeller](https://github.com/mammothb/symspellpy),

```bash
pip install symspellpy
```

```python
def symspell(
    max_edit_distance_dictionary: int = 2,
    prefix_length: int = 7,
    term_index: int = 0,
    count_index: int = 1,
    top_k: int = 10,
    **kwargs
):
    """
    Train a symspell Spell Corrector.

    Returns
    -------
    result: malaya.spell.Symspell class
    """
```

In [11]:
symspell_corrector = malaya.spell.symspell()

#### To correct a word

```python
def correct(self, word: str, **kwargs):
    """
    Most probable spelling correction for word.

    Parameters
    ----------
    word: str

    Returns
    -------
    result: str
    """
```

In [12]:
symspell_corrector.correct('bntng')

'bintang'

In [13]:
symspell_corrector.correct('kerajaan')

'kerajaan'

In [14]:
symspell_corrector.correct('mknn')

'makanan'

#### List possible generated words

```python
def edit_step(self, word):
    """
    Generate candidates given a word.

    Parameters
    ----------
    word: str

    Returns
    -------
    result: List[str]
    """
```

In [15]:
symspell_corrector.edit_step('mrh')

{'marah': 12684.0,
 'merah': 21448.5,
 'arah': 15066.5,
 'darah': 10003.0,
 'mara': 7504.5,
 'malah': 7450.0,
 'zarah': 3753.5,
 'murah': 3575.5,
 'barah': 2707.5,
 'march': 2540.5,
 'martha': 390.0,
 'marsha': 389.0,
 'maratha': 88.5,
 'marcha': 22.5,
 'karaha': 13.5,
 'maraba': 13.5,
 'varaha': 11.5,
 'marana': 4.5,
 'marama': 4.5}

#### To correct a sentence

```python
def correct_text(self, text: str):
    """
    Correct all the words within a text, returning the corrected text.

    Parameters
    ----------
    text: str

    Returns
    -------
    result: str
    """
```

In [16]:
symspell_corrector.correct_text(string1)

'kerajaan patut bagi pencen awal saat kepada warga emas supaya emosi'

In [17]:
symspell_corrector.correct_text(string2)

'Husein suka makan ayam dapat kampung Jawa'

In [18]:
symspell_corrector.correct_text(string3)

'Melayu malas ni narration dia sama sahaja macam men are trash. True to some, false to some.'

In [19]:
symspell_corrector.correct_text(string4)

'Tapi tak fikir ke bahaya perpetuate maathai macam itu. Nanti kalau ada hiring discrimination despite your good qualifications because of your race tahu pula marah. Your kids will be victims of that too.'

In [20]:
symspell_corrector.correct_text(string5)

'DrM cerita Melayu malas semenjak saya kat University (early 1980s) and now as saya am edging towards retirement in 4-5 aras time after a career of being an Engineer, Project Manager, General Manager'

In [21]:
symspell_corrector.correct_text(string6)

'boleh bintang dalam kelas malaya saya, nanti mintalah'

### List available Transformer models

We use custom spelling augmentation,

1. `replace_similar_consonants`

- mereka -> nereka

2. `replace_similar_vowels`

- suka -> sika

3. `socialmedia_form`

- suka -> ska

4. `vowel_alternate`

- singapore -> sngpore
- kampung -> kmpng

In [4]:
malaya.spell.available_transformer()

INFO:root:tested on 10k test set.


,Size (MB),Quantized Size (MB),WER,Suggested length
small-t5,355.6,195.0,0.015625,256.0
tiny-t5,208.0,103.0,0.023712,256.0
super-tiny-t5,81.8,27.1,0.038001,256.0


### Load Transformer model

```python
def transformer(model: str = 'small-t5', quantized: bool = False, **kwargs):
    """
    Load a Transformer Spell Corrector.

    Parameters
    ----------
    model : str, optional (default='small-t5')
        Model architecture supported. Allowed values:

        * ``'small-t5'`` - T5 SMALL parameters.
        * ``'tiny-t5'`` - T5 TINY parameters.
        * ``'super-tiny-t5'`` - T5 SUPER TINY parameters.

    quantized : bool, optional (default=False)
        if True, will load 8-bit quantized model.
        Quantized model not necessary faster, totally depends on the machine.

    Returns
    -------
    result: malaya.model.t5.Spell class
    """
```

In [3]:
t5 = malaya.spell.transformer(model = 'tiny-t5')

#### Predict using greedy decoder

```python
def greedy_decoder(self, strings: List[str]):
    """
    spelling correction for strings.

    Parameters
    ----------
    strings: List[str]

    Returns
    -------
    result: List[str]
    """
```

In [6]:
t5.greedy_decoder([string1])

['kerajaan patut bagi pencen awal skt kpd warga emas supaya emosi']

In [8]:
t5.greedy_decoder([string2])

['Husein suka makan ayam dekat kampung Jawa']

In [7]:
t5.greedy_decoder([string3])

['Melayu malas ni narration dia sama je macam men are trash . True to some , false to some .']